In [7]:
import pandas as pd
import numpy as np
import re

In [8]:
file_path = "C:/Users/HP/Desktop/CA2 ML/tweets.csv"

# Read the CSV file using pandas
try:
    df = pd.read_csv(file_path, encoding='latin1', sep='\t', quoting=1)  # Use sep='\t' for tab separation
except Exception as e:
    print(f"Error reading file: {e}")
# Remove leading and trailing whitespaces from all columns
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Print the DataFrame
df.sample(10)


,id,keyword,location,text,target
5856,5856,flood,"Wake County, North Carolina",Donald Trump told Ingraham that CEOs want only...,0
10130,10130,threat,NaN,"play granblue, this is a threat or i will carr...",0
6323,6323,heat%20wave,"Valhala, Rising",Oshey! Heat wave ???????? Ps: IFB https://t.co...,0
134,134,airplane%20accident,NaN,And anyone who believes that incident was an a...,0
6892,6892,injury,United Kingdom,Research suggests you should stick to your nat...,0
4860,4860,emergency%20services,"Oxford, England",For those of you who like #Rugby - are offerin...,0
3003,3003,danger,NaN,"With William Barr, Opus Dei comes to Washingto...",0
4956,4956,engulfed,NaN,This is so hot and sexy. Look at their size di...,0
6940,6940,inundated,The Great White North,"Her motto is ""Fight"" Voters should really thin...",0
7429,7429,mayhem,"Massachusetts, USA","Frank, then why do you need to read Dr Currys...",0


In [9]:
df2=pd.read_csv("C:/Users/HP/Desktop/CA2 ML/tweets2.csv")
df2

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [10]:
df = pd.concat([df, df2], ignore_index=True)

In [11]:
df.shape

(18983, 5)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18983 entries, 0 to 18982
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        18983 non-null  int64 
 1   keyword   18922 non-null  object
 2   location  13032 non-null  object
 3   text      18983 non-null  object
 4   target    18983 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 741.6+ KB


In [13]:
df['target'].value_counts()

target
0    13598
1     5385
Name: count, dtype: int64

## Data cleaning

In [14]:
keyword=df['keyword'].value_counts()
keyword.head(20)

keyword
thunderstorm     126
flattened        122
fear             120
fatalities       119
mass%20murder    119
stretcher        119
engulfed         118
deluge           118
derailment       118
drowning         117
collision        116
drown            115
deaths           114
derailed         114
sunk             113
damage           113
sirens           112
obliterate       111
body%20bags      110
attack           110
Name: count, dtype: int64

In [15]:
pattern = r'[^a-zA-Z\s,]'

# Clean the 'location' column
df['keyword'] = df['keyword'].str.replace(pattern, '', regex=True)

In [16]:
print(df['keyword'].isnull().sum())


61


In [17]:
df['keyword'] = df['keyword'].fillna('empty')  # Fill NaN with an empty string


In [18]:
import urllib.parse
df['keyword'] = df['keyword'].apply(urllib.parse.unquote)
df[['keyword']].value_counts().head(20)

keyword     
thunderstorm    126
flattened       122
fear            120
massmurder      119
fatalities      119
stretcher       119
engulfed        118
deluge          118
derailment      118
drowning        117
collision       116
drown           115
deaths          114
derailed        114
sunk            113
damage          113
sirens          112
obliterate      111
bodybags        110
attack          110
Name: count, dtype: int64

In [19]:
import nltk
nltk.download('wordnet')  # For lemmatization
nltk.download('punkt')     # For tokenization


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()

# Function to lemmatize words in the keyword column
def lemmatize_keywords(keywords):
    tokens = word_tokenize(keywords)
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized)

# Apply lemmatization to the keyword column
df['keyword'] = df['keyword'].apply(lemmatize_keywords)
df['keyword'].value_counts().head(20)

keyword
death           221
fatality        210
weapon          207
hostage         205
injury          183
casualty        172
flood           172
siren           162
wildfire        156
thunderstorm    126
flattened       122
fear            120
stretcher       119
massmurder      119
engulfed        118
derailment      118
deluge          118
drowning        117
collision       116
drown           115
Name: count, dtype: int64

In [21]:

keyword_mapping = {
    'collide': 'collision',
    'collided': 'collision',
    'crashed': 'crash',
    'crushed':'crush',
    'bioterror':'bioterrorism',
    'demolished':'demolition',
    'demolish':'demolition',
    'derail':'derailment',
    'derailed':'derailment',
    'desolate':'desolation',
    'destroy':"destruction",
    'destroyed':'destruction',
    'bloody':'blood',
    'devastated':'devastation',
    'drowned':'drown',
    'drowning':'drown',
    'deluged':'deluge',
    'emergency':'emergency service',
    'emergency plan':'emergency service',
    'evacuate':'evacuation',
    'evacuated':'evacuation',
    'explode':'explosion',
    'exploded':'explosion',
    'fatal':'fatality',
    'flooding':'flood',
    'hail':'hailstorm',
    'hazard':'hazardous',
    'hijacker':'hijack',
    'hijacking':'hijack',
    'injured':'injury',
    'inundated':'inundation',
    'mass murderer':'mass murder',
    'obliterated':'obliteration',
    'obliterate':'obliteration',
    'panicking':'panic',
    'quarantined':'quarantine',
    'rescuer':'rescue',
    'rescued':'rescue',
    'rioting':'riot',
    'screaming':'scream',
    'screamed':'scream',
    'sinking':'sink',
    'sinkhole':'sink',
    'sunk':'sink',
    'storm':'snowstorm',
    'suicide bombing':'suicide bomb',
    'suicide bomber':'suicide bomb',
    'survivor':'survive',
    'survived':'survive',
    'terrorist':'terrorism',
    'thunder':'thunderstorm',
    'trapped':'trap',
    'traumatised':'trauma',
    'wild fire':'wildfire',
    'wounded':'wound',
    'wreck':'wreckage',
    'wrecked':'wreckage',
    'annihilated':'annihilation',
    'arsonist':'arson',
    'attacked':'attack',
    'blazing':'blaze',
    'bleeding':'blood',
    'bloody':'blood',
    'body bagging':'body bag',
    'bombed':'bomb',
    'bombing':'bomb',
    'building burning':'building on fire',
    'burned':'burn',
    'burning':'burn',
    'catastrophic':'catastrophe',
    'collapsed':'collapse',
    'dead':'death',
    'electrocuted':'electrocute',
    'blown up':'blew up',
    'radiationemergency':'radiation emergency',
    'burningbuildings':'burning buildings'
   
}

df['keyword'] = df['keyword'].replace(keyword_mapping)
df['keyword'].value_counts()
#print(df['keyword'].unique())

keyword
derailment             329
drown                  329
collision              315
death                  311
blood                  296
                      ... 
rainstorm               45
radiation emergency     41
tsunami                 40
burning buildings       37
detonation              32
Name: count, Length: 152, dtype: int64

In [22]:
df['location']=df['location'].str.strip()

In [23]:
pattern = r'[^a-zA-Z\s,]'
df['location'] = df['location'].str.replace(pattern, '', regex=True)


In [24]:
df['location']=df['location'].str.title()
df['location']=df['location'].str.strip()

In [25]:
df['location'].value_counts().head(30)

location
                           299
Usa                        169
United States              151
London                     124
Uk                         115
India                      106
Australia                  105
London, England             92
New York                    88
Los Angeles, Ca             75
United Kingdom              73
Nigeria                     71
Canada                      66
California, Usa             62
Worldwide                   59
Washington, Dc              56
Earth                       52
Chicago, Il                 47
Lagos, Nigeria              43
Ireland                     43
England, United Kingdom     42
South Africa                38
Atlanta, Ga                 38
Mumbai                      38
New York, Ny                37
Everywhere                  34
Melbourne, Victoria         34
Sydney                      33
,                           33
Kenya                       33
Name: count, dtype: int64

In [26]:
location_mapping={
    'Usa':'United States',
    'Uk':"United Kingdom",
    'Bharat':'India',
    'Earth':'Unknown',
    'Everywhere':'Unknown',
    'Home':'Unknown',
    ',':'Unknown',
    'Worldwide':'Unknown',
    'England':'United Kingdom',
    'In this world':'Unknown',
    'Us':'United States',
    'Male':'Unknown',
    'City of gold':'Dubai',
    'In your head':'Unknown',
    'Sheher':'Unknown',
    'Mxico':'Mexico',
    'Ot Or Nothing':'Unknown',
    'In Your Heart':'Unknown',
    'Hell':'Unknown',
    'Httpsjereneroseweeblycom':'Unknown',
    'In My Own Dreams':'Unknown'

}
df['location'] = df['location'].replace(location_mapping)

In [27]:
# Clean the location column and fill NaN with 'Unknown'
df['cleaned_location'] = (
    df['location']
    .replace(['', np.nan], np.nan)  # Replace empty strings with NaN
    .str.replace(r'[^a-zA-Z\s,]', '', regex=True)  # Remove special characters
    .str.strip()  # Remove leading and trailing whitespace
)

df.drop('cleaned_location',axis=1,inplace=True)

df['location'] = df['location'].replace("", pd.NA)
df.loc[(df['location'].str.len() > 25) | (df['location'].str.len() == 1) , 'location'] = 'Unknown'

# After cleaning, fill NaN values
df['location'] = df['location'].fillna("Unknown")
df['location'] = df['location'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [28]:
df['location'].value_counts().head(20)

location
Unknown                    7341
United States               335
United Kingdom              211
London                      124
India                       110
Australia                   105
London, England              92
New York                     88
Los Angeles, Ca              75
Nigeria                      71
Canada                       66
California, Usa              62
Washington, Dc               56
Chicago, Il                  47
Lagos, Nigeria               43
Ireland                      43
England, United Kingdom      42
Atlanta, Ga                  38
Mumbai                       38
South Africa                 38
Name: count, dtype: int64

## Text preprocessing

In [29]:
pattern = r'[^a-zA-Z0-9\s,.#]'

# Clean the 'location' column
df['text'] = df['text'].str.replace(pattern, '', regex=True)

In [30]:

def remove_links(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
df['text'] = df['text'].apply(remove_links)


# Strip leading and trailing whitespace
df['text'] = df['text'].str.strip()

In [31]:
df.drop("id",axis=1,inplace = True)

In [32]:
import tensorflow as tf

In [33]:
X=df.drop('target',axis=1)
y=df['target']

In [34]:
messages=X.copy()
voc_size=8000

In [35]:
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()

    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

def remove_all_single_chars(text):
    return re.sub(r'\b\w\b', '', text)

# Apply the function to the corpus and also remove extra spaces
corpus = [re.sub(r'\s+', ' ', remove_all_single_chars(sentence)).strip() for sentence in corpus]



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
corpus

['communal violence bhainsa telangana stone pelted muslim house house vehicle set ablaze',
 'telangana section imposed bhainsa january clash erupted two group january po',
 'arsonist set car ablaze dealership',
 'arsonist set car ablaze dealership',
 'lord jesus love brings freedom pardon fill holy spirit set heart ablaze',
 'child chinese tweet would gone viral social medium would ablaze snl would made racist',
 'several house set ablaze ngemsibaa village oku sub division north west region cameroon',
 'asansol bjp office salanpur village set ablaze last night bjp alleged tmc behind incident police',
 'national security minister kan dapaahs side chic set internet ablaze latest powerful video',
 'creature who soul longer clarent blue ablaze thing carrying memory memory',
 'image showing havoc caused cameroon military torched house oku shameless military reported',
 'social medium went banana chuba hubbard announced monday evening plan return okstate',
 'hausa youth set area office apapa

## Sentiment analysis

In [37]:
from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    return 'positive' if polarity > 0 else 'negative' if polarity < 0 else 'neutral'

df['predicted_sentiment'] = df['text'].apply(get_sentiment)



In [39]:
# Display the results
df[['text', 'predicted_sentiment']]

,text,predicted_sentiment
0,"Communal violence in Bhainsa, Telangana. Stone...",neutral
1,Telangana Section 144 has been imposed in Bhai...,neutral
2,Arsonist sets cars ablaze at dealership,neutral
3,Arsonist sets cars ablaze at dealership,neutral
4,"Lord Jesus, your love brings freedom and pardo...",positive
...,...,...
18978,Two giant cranes holding a bridge collapse int...,neutral
18979,ariaahrary TheTawniest The out of control wild...,positive
18980,M1.94 0104 UTC5km S of Volcano Hawaii.,neutral
18981,Police investigating after an ebike collided w...,negative


## Topic modelling for theme extraction

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

lda = LatentDirichletAllocation(n_components=20, random_state=42)
lda.fit(tfidf_matrix)

terms = tfidf_vectorizer.get_feature_names_out()
for index, topic in enumerate(lda.components_):
    print(f"Topic {index + 1}:")
    print([terms[i] for i in topic.argsort()[-12:]])

Topic 1:
['drowning', 'costlier', 'refugio', 'battle', 'lvl', 'backup', 'projected', 'id', 'bigger', 'like', 'spill', 'oil']
Topic 2:
['got', 'year', 'japan', 'detonate', 'anniversary', 'pick', 'like', 'im', 'hiroshima', 'hell', 'bomb', 'atomic']
Topic 3:
['australia', 'fuck', 'get', 'building', 'people', 'change', 'forest', 'death', 'like', 'one', 'wild', 'fire']
Topic 4:
['electrocute', 'abc', 'siren', 'like', 'northern', 'latest', 'home', 'screaming', 'razed', 'california', 'wildfire', 'im']
Topic 5:
['evacuated', 'rescue', 'set', 'iran', 'sue', 'disease', 'fire', 'affected', 'fatal', 'outbreak', 'legionnaire', 'family']
Topic 6:
['eyewitness', 'need', 'record', 'found', 'new', 'crushed', 'news', 'time', 'drought', 'light', 'wave', 'heat']
Topic 7:
['demolish', 'market', 'im', 'brace', 'triumphant', 'gove', 'school', 'upheaval', 'five', 'year', 'heart', 'party']
Topic 8:
['flood', 'disaster', 'like', 'amp', 'nuclear', 'get', 'airport', 'watch', 'minute', 'swallowed', 'sandstorm', 'h

In [47]:
topic_distribution = lda.transform(tfidf_matrix)
dominant_topic = np.argmax(topic_distribution, axis=1)

df = pd.DataFrame({'text': corpus, 'dominant_topic': dominant_topic})

topic_names = {
    1: "Environmental Disasters and Oil Spills",
    2: "Atomic Bombing and Hiroshima",
    3: "Australian Wildfires and Climate Change",
    4: "Ablaze and Emergency Response",
    5: "Evacuations and Disease Outbreaks",
    6: "Natural Disasters and Heat Waves",
    7: "Political and Social Upheaval",
    8: "Floods and Sandstorms",
    9: "Severe Storms and Infrastructure Collapse",
    10: "Severe Weather Alerts",
    11: "Train Accidents and Derailments",
    12: "Earthquakes and Aftershocks",
    13: "Fires and Fatalities",
    14: "Plane Crash and Malaysian Airlines MH370",
    15: "Natural Disasters and Tornadoes",
    16: "Rescue and Migrant Crisis",
    17: "Transportation Accidents and Structural Failures",
    18: "Urban Violence and Drunk Driving",
    19: "Volcano Eruptions and Natural Hazards",
    20: "Terrorism and Bombings"
}


df['topic_name'] = df['dominant_topic'].map(topic_names)


In [48]:
df

,text,dominant_topic,topic_name
0,communal violence bhainsa telangana stone pelt...,7,Political and Social Upheaval
1,telangana section imposed bhainsa january clas...,5,Evacuations and Disease Outbreaks
2,arsonist set car ablaze dealership,4,Ablaze and Emergency Response
3,arsonist set car ablaze dealership,4,Ablaze and Emergency Response
4,lord jesus love brings freedom pardon fill hol...,12,Earthquakes and Aftershocks
...,...,...,...
18978,two giant crane holding bridge collapse nearby...,8,Floods and Sandstorms
18979,ariaahrary thetawniest control wild fire calif...,4,Ablaze and Emergency Response
18980,utc km volcano hawaii,16,Rescue and Migrant Crisis
18981,police investigating ebike collided car little...,1,Environmental Disasters and Oil Spills
